---

## 🧠 Objetivo do Script

Esse script carrega títulos de notícias de um arquivo `.xlsx`, utiliza a API **Gemini (via Google GenAI)** para classificá-los em três categorias (`previsão`, `aconteceu` e `histórico`) e salva os resultados em um novo arquivo Excel. Também oferece suporte à reexecução parcial com **backup automático**.

---

## 🔧 Requisitos

* Python 3.x
* Bibliotecas Python:

  * `pandas`
  * `python-dotenv`
  * `google-genai`
  * `openpyxl`
* Um arquivo `.env` com a variável:

  ```
  GEMINI_API_KEY=your_api_key
  ```

---

## 🚀 Como Usar

1. Crie um arquivo `.env` com sua chave da API Gemini:

   ```env
   GEMINI_API_KEY=sua_chave_aqui
   ```

2. Execute o script:

   * Ao rodar, ele solicitará o nome do arquivo de entrada (deve estar dentro da pasta `./planilhas/`).
   * O script tentará carregar um backup de classificações (`backup_titles.json`). Se existir, você pode escolher usá-lo para evitar retrabalho.

3. Ao final, os dados classificados serão salvos em:

   ```bash
   ./planilhas/noticias_classificadas.xlsx
   ```

---

## 📁 Estrutura de Arquivos Esperada

```plaintext
.
├── .env                      # Contém a chave da API Gemini
├── planilhas/
│   ├── nome_do_arquivo.xlsx  # Arquivo com colunas "title", "link", "data"
│   └── noticias_classificadas.xlsx  # Resultado final
├── backup_titles.json        # Backup automático da classificação
```

---

## 🧩 Classificações Possíveis

| Classificação | Significado                         |
| ------------- | ----------------------------------- |
| `previsão`    | Eventos que ainda não aconteceram   |
| `aconteceu`   | Eventos recentes já ocorridos       |
| `historico`   | Eventos do passado sem efeito atual |

---

## 💡 Como Funciona

1. **Limpeza dos títulos**:

   * Remove asteriscos e espaços desnecessários.

2. **Divisão em grupos**:

   * Agrupa os títulos em blocos de 200 a 300 entradas para envio à API.

3. **Classificação via Gemini**:

   * Envia os blocos com instruções específicas de formatação e classificação.

4. **Verificação de cobertura**:

   * Identifica títulos que não foram classificados corretamente e tenta novamente.

5. **Salvar resultado**:

   * Salva os dados classificados em Excel e também em JSON para uso posterior como backup.

---


In [ ]:
try:
    import os
    import math
    import pandas as pd
    import json
    from dotenv import load_dotenv
    from google import genai
except:
    print("Configurando ambiente")
    
    os.system("pip install --upgrade google-genai python-dotenv")

    from dotenv import load_dotenv
    from google import genai
    
load_dotenv()

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

def generateContent(prompt):
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,   
    )
    return response.text

def classifyTitles(data):
    return generateContent("Vou te passar titulos de noticias e quero que você classifique simplesmente como \"previsão\", para coisas que ainda não aconteceram, \"aconteceu\" para coisas que já aconteceram e \"historico\" para coisas que já aconteceram e não vão mais acontecer. Não quero explicações nem que mude os titulos, apenas a classificação. Exemplo: \"Amanhã vai chover em SC\" = previsão. Exemplo 2: \"Ontem choveu em SC\" = aconteceu. Exemplo 3: \"Em 2000 choveu em SC\" = historico. A formatação devera ser a seguinte <Titulo da notícia exatamente como foi enviado> = <Classificação>\n Titulos: " + "\n".join(data))

def dispatchTitles(articles, groupSize=300):
    titleClassification = []

    for i in range(0, math.ceil(len(articles)/groupSize)):
        start = i * groupSize
        end = (i + 1) * groupSize
        
        payload = classifyTitles([str(pos+start)+"."+article for pos,article in enumerate(articles[start:end])]).split("\n")[:-1]
        payload = [{x.split(" = ")[0].replace("*","").strip() : x.split(" = ")[1]} for x in payload]
        print(payload)
        print(len(payload))
        titleClassification += payload
        
        print(f"{min(end,len(articles))} de {len(articles)} classificações concluídas")

    return titleClassification

def dispatchRemainingTitles(articles, indexes, groupSize=200):
    titleClassification = []

    for i in range(0, math.ceil(len(indexes)/groupSize)):
        start = i * groupSize
        end = (i + 1) * groupSize
        
        selectedArticles = [articles[pos] for pos in indexes[start:end]]
        
        payload = classifyTitles([str(indexes[pos+start])+"."+article for pos,article in enumerate(selectedArticles)]).split("\n")[:-1]
        payload = [{x.split(" = ")[0].replace("*","").strip() : x.split(" = ")[1]} for x in payload]
        print(payload)
        print(len(payload))
        titleClassification += payload
        
        print(f"{min(end,len(indexes))} de {len(indexes)} classificações concluídas")

    print(titleClassification)
    return titleClassification


arquivoDeNoticias = input("Qual arquivo você quer classificar: ")
articles = pd.read_excel(f'./planilhas/{arquivoDeNoticias}')

#cleaning tht titles
articles["title"] = articles["title"].apply(lambda x: x.replace(" *", "").replace("*", "").replace("* ", "").replace("  ", " ").replace("  ", " ").replace("  ", " ").replace(u'\xa0', u' ').strip())

#Crate lookup table for title and index
lookupArticleIndex = {x:[] for x in list(articles["title"])}
[lookupArticleIndex[x].append(i) for i, x in enumerate(list(articles["title"]))]

print(lookupArticleIndex)

useBackup = (True if input("Você quer usar o backup de titulos? [sim/não]") == "sim" else False) if os.path.exists("backup_titles.json") else False

if (useBackup):
    print("Lendo backup de titulos...")
    with open("backup_titles.json", "r", encoding="utf-16") as arq:
        classifiedTitles = json.loads(arq.read())
else:
    classifiedTitles = dispatchTitles(list(articles["title"]))

    with open("backup_titles.json", "w", encoding="utf-16") as arq:
        arq.write(json.dumps(classifiedTitles))

coverage = set(map(lambda x: int(list(x.keys())[0].split(".")[0]) if "." in list(x.keys())[0][:5] else -1, classifiedTitles))
print("Not covered indexes:")
notCovered = list(set(range(len(list(articles["title"])))).difference(coverage))
print(notCovered)

while len(notCovered) > 0:
    classifiedTitles += dispatchRemainingTitles(list(articles["title"]), notCovered)
    
    print(classifiedTitles[-668:])
    
    coverage = set(map(lambda x: int(list(x.keys())[0].split(".")[0]) if "." in list(x.keys())[0][:7] else -1, classifiedTitles))
    print("Not covered indexes:")
    notCovered = list(set(range(len(list(articles["title"])))).difference(coverage))
    print(notCovered)


classifiedNews = []

for title in classifiedTitles:
    idx = int(list(title.keys())[0].split(".")[0])
    
    classifiedNews.append({
        "title": articles["title"][idx],
        "link": articles["link"][idx],
        "data": articles["data"][idx],
        "classificacao": list(title.values())[0]
    })

print("Salvando em planilha...")
df = pd.DataFrame(classifiedNews, columns=classifiedNews[0].keys())
df.to_excel("./planilhas/noticias_classificadas.xlsx", index=False)
print("Salvo com sucesso!")